In [1]:
# State

In [1]:
import pickle
import time
from typing import List
import datetime as dt
from pathlib import Path

import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px

from tqdm import tqdm

from quara.data_analysis import data_analysis, physicality_violation_check, report
from quara.objects.composite_system import CompositeSystem
from quara.objects.elemental_system import ElementalSystem
from quara.objects.matrix_basis import get_normalized_pauli_basis
from quara.objects.povm import (
    Povm,
    get_x_measurement,
    get_y_measurement,
    get_z_measurement,
)
from quara.objects.qoperation import QOperation
from quara.objects.state import State, get_z0_1q, get_z1_1q, get_x0_1q
from quara.protocol.qtomography.standard.standard_qst import StandardQst
from quara.protocol.qtomography.standard.linear_estimator import LinearEstimator
from quara.protocol.qtomography.standard.projected_linear_estimator import (
    ProjectedLinearEstimator,
)

In [2]:
%reload_ext autoreload
%autoreload 2

In [3]:
start_all = time.time()

In [4]:
# setup system
e_sys = ElementalSystem(0, get_normalized_pauli_basis())
c_sys = CompositeSystem([e_sys])

povm_x = get_x_measurement(c_sys)
povm_y = get_y_measurement(c_sys)
povm_z = get_z_measurement(c_sys)
tester_objects = [povm_x, povm_y, povm_z]

In [5]:
# Case 1:
# true_object = get_z0_1q(c_sys)
# true_object.vec

# Case 2:
# vec = np.array([1/np.sqrt(2), 1/np.sqrt(6), 1/np.sqrt(6), 1/np.sqrt(6)],dtype=np.float64)
# true_object = State(c_sys, vec)

# Case 3:
vec = np.array([1 / np.sqrt(2), 0, 0, 0], dtype=np.float64)
true_object = State(c_sys, vec)

In [11]:
num_data = [100, 1000, 10000]
n_rep = 100

case_name_list = [
    "LinearEstimator(True)",
    "LinearEstimator(False)",
    "ProjectedLinearEstimator(True)",
    "ProjectedLinearEstimator(False)",
]

seed = 777
qtomography_list = [
    StandardQst(tester_objects, on_para_eq_constraint=True, seed=seed),
    StandardQst(tester_objects, on_para_eq_constraint=False, seed=seed),
    StandardQst(tester_objects, on_para_eq_constraint=True, seed=seed),
    StandardQst(tester_objects, on_para_eq_constraint=False, seed=seed),
]
para_list = [True, False, True, False]

estimator_list = [
    LinearEstimator(),
    LinearEstimator(),
    ProjectedLinearEstimator(),
    ProjectedLinearEstimator(),
]

estimation_results_list = []
elapsed_times = []

for i, name in enumerate(case_name_list):
    qtomography = qtomography_list[i]
    estimator = estimator_list[i]
    
    start = time.time()
    print(f"Case {i}: {name}")
    print(f"Parametorization: {para_list[i]}")
    print(f"Type of qtomography: {qtomography.__class__.__name__}")
    print(f"Estimator: {estimator.__class__.__name__}")

    estimation_results = data_analysis.estimate(
       qtomography=qtomography,
       true_object=true_object,
       num_data=num_data,
       estimator=estimator,
       iteration=n_rep,
    )
    estimation_results_list.append(estimation_results)
    
    elapsed_time = time.time() - start
    print("elapsed_time:{0}".format(elapsed_time / 60) + "[min]\n")
    elapsed_times.append(elapsed_time)

  1%|          | 1/100 [00:00<00:12,  7.64it/s]

Case 0: LinearEstimator(True)
Parametorization: True
Type of qtomography: StandardQst
Estimator: LinearEstimator


  1%|          | 1/100 [00:00<00:14,  6.98it/s]

elapsed_time:0.17056113084157307[min]

Case 1: LinearEstimator(False)
Parametorization: False
Type of qtomography: StandardQst
Estimator: LinearEstimator


  1%|          | 1/100 [00:00<00:12,  7.87it/s]

elapsed_time:0.19787423610687255[min]

Case 2: ProjectedLinearEstimator(True)
Parametorization: True
Type of qtomography: StandardQst
Estimator: ProjectedLinearEstimator


  1%|          | 1/100 [00:00<00:14,  6.82it/s]

elapsed_time:0.17365981737772623[min]

Case 3: ProjectedLinearEstimator(False)
Parametorization: False
Type of qtomography: StandardQst
Estimator: ProjectedLinearEstimator


100%|██████████| 100/100 [00:10<00:00,  9.93it/s]

elapsed_time:0.16802794933319093[min]



In [12]:
data_analysis.make_mses_graph_estimation_results(
    estimation_results_list,  # 「EstimationResultのリスト」のリスト
    case_name_list,  # ケース名のリスト。ここに指定した文字列が凡例に表示される
    true_object
)

In [13]:
report.export_report("sample_qst_report.pdf",
    estimation_results_list,   # 「EstimationResultのリスト」のリスト
    case_name_list,  # ケース名（この名前がMSEなどの凡例に表示される）
    estimator_list,  # 推定量のリスト
    true_object,  # True Object
    tester_objects,   # Tester Objectのリスト.
    seed=seed,  # 推定で使ったseed（オプション）
    computation_time=sum(elapsed_times) # 処理時間の合計（オプション）
)

100%|██████████| 100/100 [00:00<00:00, 122282.92it/s]

​Generating table of computation time ...
​Generating table of experimental conditions ...
Generating case list ...
​​Generating MSE of empirical distributions blocks ...


​​Generating consictency test blocks ...
​Generating a graph for MSE ...


/Users/tomoko/project/rcast/workspace/quara/venv/lib/python3.8/site-packages/numpy/core/_methods.py:216: RuntimeWarning:

Degrees of freedom <= 0 for slice

/Users/tomoko/project/rcast/workspace/quara/venv/lib/python3.8/site-packages/numpy/core/_methods.py:209: RuntimeWarning:

invalid value encountered in double_scalars



​​Generating physicality violation test blocks ...


100%|██████████| 100/100 [00:00<00:00, 4739.11it/s]


Converting to PDF report ...
​Deleting temporary files ...
Completed to export pdf. (sample_qst_report.pdf)


In [9]:
report.export_report?